Anggota:
*   Yohanes Yordan Tjahjono
*   Samuel Christo
*   Muhammad Dira Kurnia

*   Pamella Cathryn
*   Christian August


In [ ]:
#import library
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

In [ ]:
#inisialisasi variabel
EMPTY = 0
TREE = 1
BURNING = 2
burnProbability = 0.4
time = 30

**Kondisi Awal Hutan**<br>
Hutan terdiri dari sel berukuran 17x17<br>
Setiap sel mengandung sebuah pohon<br>
Pada sel paling tengah, ujung kiri atas, dan ujung kanan bawah, pohon kebakar

In [ ]:
#Inisialisasi kondisi hutan
def hutanAwal():
  hutan = np.zeros((17, 17))
  for i in range(17):
    for j in range(17):
      if (i==0 and j==0) or (i==16 and j==16) or (i==8 and j==8):
        hutan[i][j] = BURNING
      else:
        hutan[i][j] = TREE
  return hutan

In [ ]:
#spreading function (ketetanggaan von Neumann)
def spread(site, N, E, S, W):
  if (site == EMPTY):
    newSite = EMPTY
  else:
    if (site == BURNING):
      newSite = EMPTY
    else:
      if (N == BURNING or E == BURNING or S == BURNING or W == BURNING):
        x=np.random.rand(1)
        if x < burnProbability:
          newSite = BURNING
        else:
          newSite = TREE
      else:
        newSite = TREE
  return newSite

**Periodic Boundary Condition**<br>
Extend the north boundary with a ghost row that is a copy of the original south boundary row, and extend the south boundary with a copy of the original north boundary row. Similarly, expand the column boundaries on the east and west sides

In [ ]:
# menambah 1 baris/kolom ke setiap arah pada grid (periodic boundary condition)
def periodicLat(lat):
  latNS = np.row_stack((lat[-1], lat, lat[0]))
  return np.column_stack((latNS[:,-1], latNS, latNS[:,0]))

**Ketetanggaan von Neumann**<br>
The von Neumann neighborhood of a site is the set of cells directly to the north, east, south, and west of the site and the site itself

In [ ]:
# hutan baru
def hutanBaru(lat):
  n = 17
  newLat = np.zeros((n, n))    
  
  for i in range(1, n + 1):
    for j in range(1, n + 1):
      site = lat[i][j]
      N = lat[i - 1][j]
      E = lat[i][j + 1]
      S = lat[i + 1][j]
      W = lat[i][j - 1]
      newLat[i - 1][j - 1] = spread(site, N, E, S, W)
  return newLat

In [ ]:
# memulai simulasi
def play(time):
  hutan = hutanAwal()
  
  grids = np.zeros((time+1, 17, 17))
  grids[0][:][:] = hutan #untuk t=0, hutan=hutanAwal()
  for i in range(1, time+1): #untuk t>0, hutan=hutanBaru
    hutanBesar = periodicLat(hutan)
    hutan = hutanBaru(hutanBesar)
    grids[i][:][:] = hutan
  return grids

In [ ]:
final=play(time)

# Menyiapkan figure, heatmap
fig = plt.figure()

# Fungsi init
def init():
  plt.clf()
  return None

# Fungsi iterasi animasi
def animate(i):
  plt.clf()
  ax = sns.heatmap(final[i-1],
                   vmin=0,
                   center=1,
                   vmax=2,
                   cmap="Reds",
                   square=True,
                   xticklabels=False,
                   yticklabels=False,
                   )
  ax.set_title(f"burnProbability={int(100*burnProbability)}% pada t={i-1}")
  return None

# Fungsi init
anim = animation.FuncAnimation(fig,
                               animate,
                               frames=range(1, time+1, 1),
                               blit=False,
                               interval=250,
                               init_func=init)

# Kode tambahan jika dijalankan di colab/jupyter
from matplotlib import rc
from IPython.display import HTML
rc('animation', html='jshtml')
anim

<Figure size 432x288 with 0 Axes>

In [ ]:
#kesimpulan hasil simulasi

counter=0 #menampung jumlah cell yang empty pada t=time
for i in range(17):
  for j in range(17):
    if final[-1][i][j]==0:
      counter+=1

print(f"Persentase hutan yang terbakar = {round(100*counter/(17*17),2)}%")

checker=0
for k in range(time):
  for i in range(17):
    for j in range(17):
      if final[k][i][j]==2:
        checker+=1
  if checker==counter:
    break
print(f"Waktu yang dibutuhkan sampai hutan habis terbakar adalah {k+1}")

Persentase hutan yang terbakar = 3.11%
Waktu yang dibutuhkan sampai hutan habis terbakar adalah 7
